# Overview


Tsim is a quantum circuit sampler that can efficiently sample from Clifford+T circuits with Pauli noise. It is based on ZX-calculus stabilizer rank decomposition and parametrized ZX diagrams, following work of [arXiv:2403.06777](https://arxiv.org/abs/2403.06777).

## Supported Gates

Tsim supports a universal gate set, together with measurement and reset instructions, and Pauli noise channels.

### Clifford Instructions

Tsim supports all Clifford instructions [supported by STIM](https://github.com/quantumlib/Stim/wiki/Stim-v1.9-Gate-Reference). Below we show the standard generating set of Clifford gates:

In [ ]:
from tsim import Circuit
c = Circuit("H 0")
print(c.to_matrix())
c.diagram("timeline-svg", height=120)

In [ ]:
c = Circuit("S 0")
print(c.to_matrix())
c.diagram("timeline-svg", height=120)

In [ ]:
c = Circuit("CNOT 0 1")
print(c.to_matrix())
c.diagram("timeline-svg", height=160)

In [ ]:
c = Circuit("T 0")
print(c.to_matrix())
c.diagram("timeline-svg", height=120)

### Non-Clifford Instructions

In addition to Clifford gates, Tsim supports the following non-Clifford gates. Note that all rotation parameters have to be defined in units of $\pi$.

Computation time and memory requirement scales exponentially with the number of non-Clifford gates.

In [ ]:
c = Circuit("R_X(0.1) 0")  # rotation around X-axis by 0.1π
print(c.to_matrix())
c.diagram("timeline-svg", height=120)


$$
R_X(\alpha) =
\left(
\begin{array}{cc}
\cos(\alpha\pi/2) & -i \sin(\alpha\pi/2) \\
-i \sin(\alpha\pi/2) & \cos(\alpha\pi/2)
\end{array}
\right)
$$

In [ ]:
c = Circuit("R_Y(0.1) 0")  # rotation around Y-axis by 0.1π
print(c.to_matrix())
c.diagram("timeline-svg", height=120)

$$
R_Y(\alpha) =
\left(
\begin{array}{cc}
\cos(\alpha\pi/2) & -\sin(\alpha\pi/2) \\
\sin(\alpha\pi/2) & \cos(\alpha\pi/2)
\end{array}
\right)
$$

In [ ]:
c = Circuit("R_Z(0.1) 0") # rotation around Z-axis by 0.1π
print(c.to_matrix())
c.diagram("timeline-svg", height=120)

$$
R_Z(\alpha) =
\left(
\begin{array}{cc}
e^{-i\alpha\pi/2} & 0 \\
0 & e^{i\alpha\pi/2}
\end{array}
\right)
$$

In [ ]:
c = Circuit("U3(0.1, 0.2, 0.3) 0")
print(c.to_matrix())
c.diagram("timeline-svg", height=120)

$$
U_3(\theta, \phi, \lambda) =
\left(
\begin{array}{cc}
\cos(\theta\pi/2) & -e^{i\lambda\pi}\sin(\theta\pi/2) \\
e^{i\phi\pi}\sin(\theta\pi/2) & e^{i(\phi+\lambda)\pi}\cos(\theta\pi/2)
\end{array}
\right)
$$

### Measurement and Reset instructions

Tsim supports all collapsing gates [supported by STIM](https://github.com/quantumlib/Stim/wiki/Stim-v1.9-Gate-Reference#collapsing-gates).

In [ ]:
c = Circuit("M 0")
c.diagram("timeline-svg", height=120)

Measurements (`M`, `MX`, `MY`, `MZ`) project the state into the measurement basis and write the resulting bit into the measurement record.

The measurement record can be used to conditionally apply Pauli gates:

In [ ]:
c = Circuit("""
M 0
CY rec[-1] 1
""")
c.diagram("timeline-svg", height=170)

The `!` operator can be used to invert the classical measurement bit that is written into the measurement record:

In [ ]:
c = Circuit("M !0")
c.diagram("timeline-svg", height=120)

The `MPP` instruction measures Pauli strings. The MPP can also be used in conjunction with the `!` operator to flip the classical measurement bit before writing it into the measurement record.

In [ ]:
c = Circuit("MPP !Z0*Z2*Z3")
c.diagram("timeline-svg", height=220)

### Noise Channels

Tsim supports all noise channels [supported by STIM](https://github.com/quantumlib/Stim/wiki/Stim-v1.9-Gate-Reference#noise-channels), except the `CORRELATED_ERROR` instruction.

The `X_ERROR(p)` instruction is a `X` instruction that is applied with probability `p`.

In [ ]:
c = Circuit("X_ERROR(0.1) 0")
c.diagram("timeline-svg", height=120)

The `PAULI_CHANNEL_1(p_x, p_y, p_z)` instruction is a `X`, `Y`, and `Z` instruction that is applied with probabilities `p_x`, `p_y`, and `p_z` respectively.

In [ ]:
c = Circuit("PAULI_CHANNEL_1(0.1, 0.2, 0.3) 0")
c.diagram("timeline-svg", height=120)

The `PAULI_CHANNEL_2` instruction takes fifteen floats specifying the disjoint probabilities of each possible Pauli pair
that can occur (except for the non-error double identity case).
The disjoint probability arguments are (in order):

p_ix,
p_iy,
p_iz,
p_xi,
p_xx,
p_xy,
p_xz,
p_yi,
p_yx,
p_yy,
p_yz,
p_zi,
p_zx,
p_zy,
p_zz

In [ ]:
c = Circuit("PAULI_CHANNEL_2(0.01, 0.01, 0.03, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01) 0 1")
c.diagram("timeline-svg", height=170)

The `DEPOLARIZE1(p)` instruction applies a randomly chosen Pauli with probability `p`.

In [ ]:
c = Circuit("DEPOLARIZE1(0.01) 0")
c.diagram("timeline-svg", height=120)

The `DEPOLARIZE2(p)` instruction applies a randomly chosen two-qubit Pauli with probability `p`.

In [ ]:
c = Circuit("DEPOLARIZE2(0.01) 0 1")
c.diagram("timeline-svg", height=170)

### Annotations

Tsim supports detector and observable annotations.

The `DETECTOR` instruction is only used in detector sampling mode and ignored otherwise. It instructs the detector sampler to record the XOR of classical outcomes of specified measurement bits.

In [ ]:
c = Circuit("""
    M 0 0
    DETECTOR rec[-1] rec[-2]
""")
c.diagram("timeline-svg", height=150)

The `OBSERVABLE_INCLUDE` instruction is only used in observable sampling mode and ignored otherwise. It instructs the detector sampler to record the XOR of the specified measurement bits.

In [ ]:
c = Circuit("""
    M 0 0
    OBSERVABLE_INCLUDE(0) rec[-1] rec[-2]
""")
c.diagram("timeline-svg", height=150)

## Sampling

Tsim supports multiple samplers. The first is a measurement sampler. This will simply sample bits for each measurement instruction in the circuit. Detector and observable annotations will simply be ignored by this sampler.

In [ ]:
c = Circuit("""
    RX 0
    R 1
    CNOT 0 1
    M 0 1
""")
sampler = c.compile_sampler()
c.diagram("timeline-svg", height=170)

In [ ]:
sampler.sample(shots=5)

The second sampling mode is detector sampling. This will sample detector events and observable values. Detector and observable bits can always be obtained by linear transformations of the measurement bits as return by the measurement sampler.
In practice, however, it can be much more efficient to sample detector events directly.

In [ ]:
c = Circuit("""
    RX 0
    R 1
    CNOT 0 1
    M 0 1
    DETECTOR rec[-1] rec[-2]
    OBSERVABLE_INCLUDE(0) rec[-1]
""")
sampler = c.compile_detector_sampler()
c.diagram("timeline-svg", height=170)

In [ ]:
detectors, observables = sampler.sample(5, separate_observables=True)
print(detectors)
print(observables)


Finally, Tsim allows to compute probability values for target states via the `CompiledStateProbs` sampler.

In [ ]:
import numpy as np
from tsim.sampler import CompiledStateProbs
sampler = CompiledStateProbs(c)

In [ ]:
sampler.probability_of(np.array([0, 0]), batch_size=1)

In [ ]:
sampler.probability_of(np.array([0, 1]), batch_size=1)

In [ ]:
sampler.probability_of(np.array([1, 0]), batch_size=1)

In [ ]:
sampler.probability_of(np.array([1, 1]), batch_size=1)

## Detector Error Models

Tsim allows to compute detector error models from a circuit. As opposed to Stim, detectors and observables need not be deterministic.

In [ ]:
c = Circuit("""
    RX 0
    R 1
    CNOT 0 1
    DEPOLARIZE1(0.1) 0 1
    M 0 1
    DETECTOR rec[-1] rec[-2]
    DETECTOR rec[-1]
""")
c.detector_error_model()